In [1]:
import pandas as pd
import re 
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import pickle
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
# import en_core_web_sm
import nltk
import os
from nltk.tokenize import word_tokenize
import emoji

C:\Users\Annelie Schridde\AppData\Roaming\Python\Python38\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
#import sys
#sys.path.append('../')
#import functions
#exec(open('./functions.py').read())
%run functions.py

C:\Users\Annelie Schridde\AppData\Roaming\Python\Python38\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to C:\Users\Annelie
[nltk_data]     Schridde\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Annelie
[nltk_data]     Schridde\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Annelie
[nltk_data]     Schridde\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
... OK (Got response in 0.68 seconds)
Writing emoji data to C:\Users\Annelie Schridde\.demoji\co

In [3]:
df = pd.read_json('./data_n_models/one_cleaned_df.json')
de_df = df[df.language == 'de'].reset_index()
de_df.head()

,level_0,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,language,keyword,favorite_count,in_reply_to_status_id
0,0,0,793418126500000000,521,MILESmobility,Good News! Alle interessierten MitarbeiterInne...,0,"Berlin, Deutschland",05-Mar-2021,768,False,de,@zalando,NaN,NaN
1,1,1,1119231287000000128,23,pici1303,@Zalando Ich weiß dass der Schein da rein muss...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN
2,2,2,1119231287000000128,23,pici1303,@Zalando Ich hab mich nur gefragt wofür dieser...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN
3,3,3,1119231287000000128,23,pici1303,@Zalando Ich hab doch schon den Rücksendeschei...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN
4,4,4,1119231287000000128,23,pici1303,@Zalando bei der Retoure ist noch ein einzelne...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN


In [4]:
def clean_complete(tweet):
    """
    tweet: pandas series
    prepares tweets complete cleaning for further lemmatization and dering embeddings
    """
    pat = r"(\\n)|(@\w*)|((www\.[^\s]+)|(https?://[^\s]+))"
    tweet = tweet.str.replace(pat, '')

    #remove repeated charachters
    
    #replace emoticons with words
    #SMILEYS = {":-(":"sad", ":‑)":"smiley", ":-P":"playfullness", ":-/":'confused'}

    tweet = tweet.str.replace(r':-\)', ' smile')
    tweet = tweet.str.replace(r':-\(', ' sad')
    tweet = tweet.str.replace(r':-\/', ' confused')
    tweet = tweet.str.replace(r':-P', ' playfullness')

    #delete \xa
    tweet = tweet.str.replace('\xa0', '')

    tweet = tweet.str.replace('&amp', '')
    tweet = tweet.str.replace('\n', '')
    tweet = tweet.str.replace('"', '')
    #to lower case
    tweet = tweet.str.lower()

    #covert hashtags to the normal text
    tweet = tweet.str.replace(r'#([^\s]+)', r'\1')

    #delete numbers
    tweet = [strip_numeric(c) for c in tweet]

    #replacing emojies with descriptions '❤️-> red heart'
    #tweet = [demoji.replace_with_desc(c, ' ') for c in tweet]

    #delete punctuation
    tweet = [strip_punctuation(c) for c in tweet]

    #remove stop words
    tweet = [remove_stopwords(c) for c in tweet]

    #remove short words
    tweet = [strip_short(c) for c in tweet]

    #remove mult whitespaces
    tweet = [strip_multiple_whitespaces(c) for c in tweet]
    return tweet


In [5]:
def clean_vader(tweet):
    """
    tweet: pandas series
    prepares tweets for vader sentiment analysis
    """

    pat = r"(\\n)|(@\w*)|((www\.[^\s]+)|(https?://[^\s]+))"
    tweet = tweet.str.replace(pat, '')

    #replace emoticons with words
    #SMILEYS = {":-(":"sad", ":‑)":"smiley", ":-P":"playfullness", ":-/":'confused'}

    #tweet = tweet.str.replace(r':-\)', ' smile')
    #tweet = tweet.str.replace(r':-\(', ' sad')
    #tweet = tweet.str.replace(r':-\/', ' confused')
    #tweet = tweet.str.replace(r':-P', ' playfullness')

    #delete \xa
    tweet = tweet.str.replace('\xa0', '')

    tweet = tweet.str.replace('&amp', '')
    tweet = tweet.str.replace('\n', '')

    #to lower case
    #tweet = tweet.str.lower()

    #covert hashtags to the normal text
    tweet = tweet.str.replace(r'#([^\s]+)', r'\1')

    #delete numbers
    tweet = [strip_numeric(c) for c in tweet]

    #replacing emojies with descriptions '❤️-> red heart'
    #tweet = [demoji.replace_with_desc(c, ' ') for c in tweet]

    #delete punctuation
    #tweet = [strip_punctuation(c) for c in tweet]

    #remove stop words
    #tweet = [remove_stopwords(c) for c in tweet]

    #remove short words
    tweet = [strip_short(c) for c in tweet]

    #remove mult whitespaces
    tweet = [strip_multiple_whitespaces(c) for c in tweet]
    return tweet


In [6]:
#import demoji
de_df['clean'] = clean_complete(de_df.tweet)
de_df.head()

<ipython-input-4-abe84fc2115d>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(pat, '')
<ipython-input-4-abe84fc2115d>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(r':-\)', ' smile')
<ipython-input-4-abe84fc2115d>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(r':-\(', ' sad')
<ipython-input-4-abe84fc2115d>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(r':-\/', ' confused')
<ipython-input-4-abe84fc2115d>:29: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(r'#([^\s]+)', r'\1')


,level_0,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,language,keyword,favorite_count,in_reply_to_status_id,clean
0,0,0,793418126500000000,521,MILESmobility,Good News! Alle interessierten MitarbeiterInne...,0,"Berlin, Deutschland",05-Mar-2021,768,False,de,@zalando,NaN,NaN,good news alle interessierten mitarbeiterinnen...
1,1,1,1119231287000000128,23,pici1303,@Zalando Ich weiß dass der Schein da rein muss...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN,ich weiß dass der schein rein muss aber ist mi...
2,2,2,1119231287000000128,23,pici1303,@Zalando Ich hab mich nur gefragt wofür dieser...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN,ich hab mich nur gefragt wofür dieser extra co...
3,3,3,1119231287000000128,23,pici1303,@Zalando Ich hab doch schon den Rücksendeschei...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN,ich hab doch schon den rücksendeschein draufge...
4,4,4,1119231287000000128,23,pici1303,@Zalando bei der Retoure ist noch ein einzelne...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN,bei der retoure ist noch ein einzelner extra c...


In [7]:
de_df['clean_vader'] = clean_vader(de_df.tweet)
de_df.head()

<ipython-input-5-dfb30a3f3187>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(pat, '')
<ipython-input-5-dfb30a3f3187>:28: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(r'#([^\s]+)', r'\1')


,level_0,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,language,keyword,favorite_count,in_reply_to_status_id,clean,clean_vader
0,0,0,793418126500000000,521,MILESmobility,Good News! Alle interessierten MitarbeiterInne...,0,"Berlin, Deutschland",05-Mar-2021,768,False,de,@zalando,NaN,NaN,good news alle interessierten mitarbeiterinnen...,Good News! Alle interessierten MitarbeiterInne...
1,1,1,1119231287000000128,23,pici1303,@Zalando Ich weiß dass der Schein da rein muss...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN,ich weiß dass der schein rein muss aber ist mi...,Ich weiß dass der Schein rein muss aber was is...
2,2,2,1119231287000000128,23,pici1303,@Zalando Ich hab mich nur gefragt wofür dieser...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN,ich hab mich nur gefragt wofür dieser extra co...,Ich hab mich nur gefragt wofür dieser extra Co...
3,3,3,1119231287000000128,23,pici1303,@Zalando Ich hab doch schon den Rücksendeschei...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN,ich hab doch schon den rücksendeschein draufge...,Ich hab doch schon den Rücksendeschein draufge...
4,4,4,1119231287000000128,23,pici1303,@Zalando bei der Retoure ist noch ein einzelne...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN,bei der retoure ist noch ein einzelner extra c...,bei der Retoure ist noch ein einzelner extra c...


In [8]:
print(de_df.tweet[2])
print(de_df.clean[2])
print(de_df.clean_vader[2])

@Zalando Ich hab mich nur gefragt wofür dieser extra Code ist? https://t.co/e8gQcklkL7
ich hab mich nur gefragt wofür dieser extra code ist
Ich hab mich nur gefragt wofür dieser extra Code ist?


In [9]:
#Vater sentiment analyzer
#The Compound score is a metric that calculates the sum of all the lexicon ratings which have been normalized between -1(most extreme negative) and +1 (most extreme positive).

sid_obj = SentimentIntensityAnalyzer()
de_df['compound'] = [sid_obj.polarity_scores(c)['compound'] for c in de_df['clean_vader']]

In [10]:
print(de_df[['clean_vader', 'compound']].describe())

           compound
count  10605.000000
mean      -0.161764
std        0.368385
min       -0.966100
25%       -0.599400
50%        0.000000
75%        0.000000
max        0.969800


# Testing cleaning function

In [11]:
#testing
#delete links, users
pat = r"(\\n)|(@\w*)|((https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,}))"
de_df.tweet = de_df.tweet.str.replace(pat, '')
de_df.tweet[2]

<ipython-input-11-5463527a300c>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  de_df.tweet = de_df.tweet.str.replace(pat, '')


' Ich hab mich nur gefragt wofür dieser extra Code ist? '

In [12]:
#remove chars more than twice
#str.replace( /(.)\1{2,}/g, '$1$1' )

In [13]:

#testing
# #replace emoticons with words
#SMILEYS = {":-(":"sad", ":‑)":"smiley", ":-P":"playfullness", ":-/":'confused'}

de_df.tweet = de_df.tweet.str.replace(r':-\)', ' smile')
de_df.tweet = de_df.tweet.str.replace(r':-\(', ' sad')
de_df.tweet = de_df.tweet.str.replace(r':-\/', ' confused')
de_df.tweet = de_df.tweet.str.replace(r':-P', ' playfullness')

de_df.tweet[2]

<ipython-input-13-a5a60d69cb5d>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  de_df.tweet = de_df.tweet.str.replace(r':-\)', ' smile')
<ipython-input-13-a5a60d69cb5d>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  de_df.tweet = de_df.tweet.str.replace(r':-\(', ' sad')
<ipython-input-13-a5a60d69cb5d>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  de_df.tweet = de_df.tweet.str.replace(r':-\/', ' confused')


' Ich hab mich nur gefragt wofür dieser extra Code ist? '

In [14]:
#testing#delete \xa
de_df.tweet = de_df.tweet.str.replace('\xa0', '')
de_df.tweet[0]

'Good News! Alle interessierten MitarbeiterInnen der Berliner  erhalten täglich ein #Mobilitätsbudget in Höhe von 10€, das sie für Fahrten mit #MILES einlösen können.\n\nDanke an , die die ganze Aktion sponsern! '

In [15]:
#testing
de_df.tweet = de_df.tweet.str.replace('&amp', '')
de_df.tweet = de_df.tweet.str.replace('\n', '')
de_df.tweet[3140]


' Sehen wir vielleicht bald auch eine Beauty-Eigenmarken bei Zalando?'

In [16]:
#testing#to lower case
de_df.tweet = de_df.tweet.str.lower()
de_df.tweet[2]

' ich hab mich nur gefragt wofür dieser extra code ist? '

In [17]:
#testing#covert hashtags to the normal text
de_df.tweet = de_df.tweet.str.replace(r'#([^\s]+)', r'\1')
de_df.tweet[2]

<ipython-input-17-46b7def794f7>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  de_df.tweet = de_df.tweet.str.replace(r'#([^\s]+)', r'\1')


' ich hab mich nur gefragt wofür dieser extra code ist? '

In [18]:
#testing#delete numbers
de_df.tweet = [strip_numeric(c) for c in de_df.tweet]
de_df.tweet[7]

'  solche loser'

In [27]:
#testing#replacing emojies with descriptions '❤️-> red heart'
de_df.tweet = [demoji.replace_with_desc(c, ' ') for c in de_df.tweet]
de_df.tweet[9]

'zalando braucht einfach tage für ne lieferung ich bestell da nie wieder wtf schonmal von nächster tag delivery gehört ach du scheiße ihr versager'

In [28]:
##testing delete punctuation
de_df.tweet = [strip_punctuation(c) for c in de_df.tweet]
de_df.tweet[7]

'solche loser'

In [29]:
#testing
de_df.tweet = [remove_stopwords(c) for c in de_df.tweet]
de_df.tweet[7]

'solche loser'

In [30]:
#testin
de_df.tweet = [strip_multiple_whitespaces(c) for c in de_df.tweet]
de_df.tweet[7]

'solche loser'

In [31]:
def lemmatize(tweet):
    '''
    tweet: pandas series
    should be applied on the cleaned tweets to transform words to their initial base form.
    For example: suggests -> suggest, deliveries -> delivery
    '''
    nlp = spacy.load("en_core_web_sm")
    tweet = [nlp(c) for c in tweet]
    tweet = [" ".join([token.lemma_ for token in t]) for t in tweet]
    return tweet

In [44]:
import spacy
#spacy.load('en_core_web_sm')
from spacy.lang.de import German
#nlp = spacy.load("en_core_web_sm") # English?
nlp = spacy.load("de_core_news_sm") # German


In [42]:
de_df['lem'] = [nlp(c) for c in de_df.clean]

In [43]:
de_df.lem[868]

asos und zara hat eig immer

In [45]:
de_df['lemma'] = [" ".join([token.lemma_ for token in t]) for t in de_df.lem]
de_df.lemma[7]

'solche lose'

In [46]:
de_df.head()

,level_0,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,language,keyword,favorite_count,in_reply_to_status_id,clean,clean_vader,compound,lem,lemma
0,0,0,793418126500000000,521,MILESmobility,good news alle interessierten mitarbeiterinnen...,0,"Berlin, Deutschland",05-Mar-2021,768,False,de,@zalando,NaN,NaN,good news alle interessierten mitarbeiterinnen...,Good News! Alle interessierten MitarbeiterInne...,-0.7568,"(good, news, alle, interessierten, mitarbeiter...",good news all interessieren mitarbeiterinnen d...
1,1,1,1119231287000000128,23,pici1303,ich weiß dass der schein da rein muss aber ist...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN,ich weiß dass der schein rein muss aber ist mi...,Ich weiß dass der Schein rein muss aber was is...,0.0000,"(ich, weiß, dass, der, schein, rein, muss, abe...",ich weiß dass der schein rein muss aber sein m...
2,2,2,1119231287000000128,23,pici1303,ich hab mich nur gefragt wofür dieser extra co...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN,ich hab mich nur gefragt wofür dieser extra co...,Ich hab mich nur gefragt wofür dieser extra Co...,0.0000,"(ich, hab, mich, nur, gefragt, wofür, dieser, ...",ich hab sich nur fragen wofür dies extra code ...
3,3,3,1119231287000000128,23,pici1303,ich hab doch schon den rücksendeschein draufge...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN,ich hab doch schon den rücksendeschein draufge...,Ich hab doch schon den Rücksendeschein draufge...,0.0000,"(ich, hab, doch, schon, den, rücksendeschein, ...",ich hab doch schon der rücksendeschein draufge...
4,4,4,1119231287000000128,23,pici1303,bei der retoure ist noch ein einzelner extra q...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN,bei der retoure ist noch ein einzelner extra c...,bei der Retoure ist noch ein einzelner extra c...,0.0000,"(bei, der, retoure, ist, noch, ein, einzelner,...",bei der retoure sein noch einen einzeln extra ...


# Data description
- tweet: initial tweet as it was downloaded from API
- clean: tweet cleaned completely from punctuation, emojies, emoticons, stopwords, special characters, users, hashtags, links
- clean_vader: tweet partly cleaned, saving punctuation, emoticons, emojies for using library Vader to get the sentiment of the tweet
- compouns: from -1 (negative) to 1 (positive), 0 - neutral, a tweet sentiment derived fro, Vader library
- lem: intermidiate step before lemmatization
- lemma: lemmatized words (sent -> send)

# Save the dataframe

In [47]:
pickle.dump(de_df, open(r'data_n_models/de_df_labelled.pkl', 'wb'))